<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/RTOFS/AWS_download/ESPC_D_SSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

## Download `SSH` [from ESPC-D](https://data.hycom.org/datasets/ESPC-D-V02/data/archive/2025/) and plot it.

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import glob as glob
from PIL import Image

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

%matplotlib inline

In [ ]:
def plot_region(plot_data, plot_date, vName='surf_el',
                vMin=-0.3, vMax=0.8, cMap='gist_ncar'):

  fig = plt.figure(figsize=(10, 12), dpi=180)
  ax = fig.add_subplot(1,1,1,
                       projection=ccrs.PlateCarree(central_longitude=-180.0))

  ax.add_feature(cfeature.LAND, zorder=0,
                 edgecolor='k', facecolor=("lightgray"), alpha=0.7)
  ax.coastlines(alpha=0.2)

  gl = ax.gridlines(draw_labels=True)
  gl.top_labels = False
  gl.right_labels = False
  gl.xformatter = LONGITUDE_FORMATTER
  gl.yformatter = LATITUDE_FORMATTER
  gl.xlabel_style = {'size': 6}
  gl.ylabel_style = {'size': 6}

  im= ax.pcolormesh(plot_data['lon'].values,
                     plot_data['lat'].values,
                     plot_data[vName].values.squeeze(),
                     transform=ccrs.PlateCarree(),
                     vmin=vMin, vmax=vMax, cmap=cMap)

  cbar=plt.colorbar(im, ax=ax, pad=0.03, orientation='horizontal', shrink=0.85)

  if vName == 'surf_el':
    cbar.set_label("Sea Surface Height (m)")

  ax.set_title("{}".format(plot_date))

  figName= vName + '_' + plot_date + '.png'
  plt.savefig(figName, bbox_inches='tight')
  plt.close()

## User inputs

In [ ]:
url_base = "https://data.hycom.org/datasets/ESPC-D-V02/data/archive/"
fPref = "US058GCOM-OPSnce.espc-d-031-hycom_fcst_glby008_"

start_date, end_date = ["20250320", "20250321"]

fld = "ssh" #options: Sssh (steric SSH)/ice/s3z/ssh/t3z/u3z/v3z
vName = "surf_el" # name in file

hSkip=3 # plot every hour
hours = np.arange(0, 24, hSkip)

# Plot region bounds
lat_s, lat_e = [-50, 50]
lon_s, lon_e = [100, 290]

#US058GCOM-OPSnce.espc-d-031-hycom_fcst_glby008_202503 25 12_t0000_Sssh.nc"

In [ ]:
data_dates = pd.date_range(start=start_date, end=end_date)

for dd in data_dates:
  print("Downloading data for:\t", dd.strftime("%Y%m%d"))
  for hr in hours:
    url = url_base+dd.strftime("%Y/")+\
    fPref + dd.strftime("%Y%m%d") +"12"+\
     "_t" + str(hr).zfill(4)+ "_"+ fld + ".nc"
    #print(url)
    fName_save = fld+"_"+dd.strftime("%Y%m%d")+"_"+str(hr).zfill(4)+".nc"

    !wget $url -O $fName_save

In [ ]:
ds = xr.open_mfdataset(fld+'*.nc', decode_times=False)

## Plot SSH (`surf_el`) in the selected region

In [ ]:
print(f'Plotting data from:\n')
for iD, dd in enumerate(data_dates):
  for hr in hours:
    plot_date = dd.strftime("%Y%m%d")+"_"+str(hr).zfill(4)
    print(f"{plot_date}")

    # cut region
    plot_data = ds.isel(time=iD).\
    sel(lat=slice(lat_s, lat_e), lon=slice(lon_s, lon_e))

    plot_region(plot_data, plot_date)

In [ ]:
!mkdir -p figs
!mv *.png figs/

## Make a gif animation from png files

In [ ]:
png_files_path = 'figs/'
png_files = sorted( glob.glob( png_files_path + '/' + '{}_*.png'.format(vName)))
#print(fNames)

DUR, LOOP = [150, 0] # Duration in milliseconds, infinite loop

images = []
for filename in png_files:
  im = Image.open(filename)
  images.append(im)

# save as a gif
fOut = 'espc-d_{}_'.format(vName) + start_date + '_' + end_date + '.gif'
images[0].save(fOut,save_all=True,
               append_images=images[0:],
               optimize=False,
               duration=DUR, # Duration in milliseconds
               loop=LOOP) # infinite loop

print('\nSaved:\t{}\n'.format(fOut))